In [ ]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# Write a notebook to numerically integrate the function f(x)= e^(-2x)cos(10x)

### Define a function

In [ ]:
def func(x):
    return np.exp(-2*x)*np.cos(10*x)

### Define core of trapezoid method

In [ ]:
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x*h)+f(x))

In [ ]:
def trapezoid_method(f,a,b,N):
    #f is argument of integration, a is lower limit, b is upper limit, N is number of function evals
    #if N is odd, we don't need to adjust the last segment
    
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    F_int = 0.0
    
    #Perform integral with trapezoid method
    
    for i in range(0,len(x)-1,1):
        F_int += trapezoid_core(f,x[i],h)
    
    return F_int, i

## Define core of simpson's method

In [ ]:
def simpson_core(f,x,h):
    return h*(f(x) + 4*f(x+h) +f(x+2*h))/3.

### Define a wrapper to perform Simpson's method

In [ ]:
def simpson_method(f,a,b,N):
    #f is argument of integration, a is lower limit, b is upper limit, N is number of function evals
    #if N is odd, we don't need to adjust the last segment
    
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    F_int = 0.0
    
    #Perform integral with Simpson's method
    for i in range(0,len(x)-2,2):
        F_int += simpson_core(f,x[i],h)
        
    #apply Simpson's rule over last interval if N is even
    if(N%2==0):
        F_int += simpson_core(f,x[-2],0.5*h)
    
    return F_int, i

## Define Romberg Integration Core

In [ ]:
def romberg_core(f,a,b,I,i=0):
    #need diff b/t a&b
    h = b-a
    
    #and increment b/t new function evals
    dh = h/(2.**(i))
    
    #we need the cofactor and function evals
    K = h/(2.**(i+1))
    M=0.0
    
    for j in range(2**i):
        M+= f(a + 0.5*dh + j*dh)
    return K*M

## Define wrapper function for Romberg Integration

In [ ]:
def romberg_method(f,a,b,tol=1e-6):
    i=0
    imax = 1000
    
    # define an error estimate set to a large value
    delta = 100*np.fabs(tol)
    
    I = np.zeros(imax, dtype=float)
    
    #get zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a)+f(b))
    i+=1
    
    while(delta>tol):
        #find romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        if(I[i]!=0):
            delta = np.fabs( (I[i]-I[i-1])/I[i])
        else:
            raise ValueError('Integral is zero, infinite loop stopped')
        
        if(delta>tol):
            i += 1
            
            if(i>imax):
                print("Error: Max iterations Reached")
                raise StopIteration('Stopping after iteration ',i)
    return I[i]

In [ ]:
F_int_trapezoid, i_trapezoid = trapezoid_method(func(x),0,np.pi(),1000)
F_int_simpson, i_simpson = simpson_method(func(x),0,np.pi(),1000)

print("Trapezoid: ", F_int_trapezoid, " Iterations: ", i_trapezoid)
print("Simpson's: ", F_int_simpson, " Iterations: ", i_simpson)
RI = romberg_method(func,0,1)
print("Romberg: ", RI, "Iterations: ", RI.i())